# Movie Recommender with LangChain

In [1]:
# Import libs
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema import BaseOutputParser

# sk
from sk import my_sk

## Define Chain

In [4]:
# Define chatbot
model = ChatOpenAI(openai_api_key=my_sk, # my secret key
                  model="gpt-4-turbo", # best model for task
                  temperature=0)

In [6]:
# Define Prompt Template
prompt_template = """
You are an expert movie reviewer with a vast knowledge of movies across all genres and time periods.
Your task is to recommend three movies that align with the user's interests. These interests are
specified by:
- **Genre of Interest (G):** A genre they enjoy.
- **Movie of Interest (M):** A specific movie they like within that genre.
- **Age (A):** A classifier for new versus older films.

Use the information provided to recommend movies that align with their preferences. Recommendations
should vary slightly in style, tone, or subgenres within the specified genre. Justify your recommendations
based on similarities to the given movie or genre, or based on unique elements of the recommendation
that align with the user’s preferences.

**Input:**
**G:** {genre}
**M:** {favorite_movie_in_genre}
**A:** {age}

**Output:**
Based on user preferences, the output should be simple and concise:

Title: [title]
Reason: [Max 100 tokens]
Trailer Link: [link]

ENSURE THAT THREE RECOMMENDATIONS ARE INCLUDED
"""

prompt = PromptTemplate(
    input_variables=["genre", "favorite_movie_in_genre", "age"],
    template=prompt_template
)

In [8]:
# Define Chain
chain = prompt | model


# LangChain allows chaining operations using the pipeline (|) operator.
# `prompt | model` creates a sequence where the prompt is executed, and its output is passed to the model.
# So, no nee to explicit have LLMChain. The new approach simplifies chaining operations and avoids the need for LLMChain.

## Use Chain

In [11]:
 # Define Inputs
genre = "action/drama"
favorite_movie_in_genre = "interstellar"
age = "new"

# run chain
output = chain.invoke({'genre': genre, 'favorite_movie_in_genre': favorite_movie_in_genre, 'age': age})

output_content = output.content

In [15]:
output_content

"Title: Inception\nReason: Directed by Christopher Nolan, like Interstellar, Inception offers a complex narrative blending action with deep psychological and philosophical themes. Its innovative storytelling and visual effects make it a compelling watch for fans of intellectually stimulating cinema.\nTrailer Link: [https://www.youtube.com/watch?v=YoHD9XEInc0](https://www.youtube.com/watch?v=YoHD9XEInc0)\n\nTitle: Arrival\nReason: Arrival, a film directed by Denis Villeneuve, explores themes of communication and time, similar to Interstellar's exploration of space and time. It combines drama and sci-fi elements with a strong emotional core, making it a thought-provoking experience.\nTrailer Link: [https://www.youtube.com/watch?v=tFMo3UJ4B4g](https://www.youtube.com/watch?v=tFMo3UJ4B4g)\n\nTitle: Gravity\nReason: Gravity is a visually stunning film that delves into the themes of survival and resilience in space, akin to Interstellar. Directed by Alfonso Cuarón, it offers breathtaking vis

In [17]:
print(output_content)

Title: Inception
Reason: Directed by Christopher Nolan, like Interstellar, Inception offers a complex narrative blending action with deep psychological and philosophical themes. Its innovative storytelling and visual effects make it a compelling watch for fans of intellectually stimulating cinema.
Trailer Link: [https://www.youtube.com/watch?v=YoHD9XEInc0](https://www.youtube.com/watch?v=YoHD9XEInc0)

Title: Arrival
Reason: Arrival, a film directed by Denis Villeneuve, explores themes of communication and time, similar to Interstellar's exploration of space and time. It combines drama and sci-fi elements with a strong emotional core, making it a thought-provoking experience.
Trailer Link: [https://www.youtube.com/watch?v=tFMo3UJ4B4g](https://www.youtube.com/watch?v=tFMo3UJ4B4g)

Title: Gravity
Reason: Gravity is a visually stunning film that delves into the themes of survival and resilience in space, akin to Interstellar. Directed by Alfonso Cuarón, it offers breathtaking visuals and a

## Adding an Output Parser

In [19]:
import re

# Define output parser
def parse_movie_output(output_content):
    # regex to capture each title, reason, and trailer link
    pattern = r"\*?\*?Title:\*?\*? (.+?)\n\*?\*?Reason:\*?\*? (.+?)\n\*?\*?Trailer Link:\*?\*? \[(https?://[^\]]+)\]"
    
    # Use re.findall to find all matches in the output content
    movies = re.findall(pattern, 
                        output_content)
    
    # Convert the list of tuples into a list of dictionaries
    parsed_movies = [{"title": movie[0], "reason": movie[1], "trailer_link": movie[2]} for movie in movies]
    
    return parsed_movies

In [21]:
parsed_movies = parse_movie_output(output_content)

for movie in parsed_movies:
    print(f"Title: {movie['title']}")
    print(f"Reason: {movie['reason']}")
    print(f"Trailer Link: {movie['trailer_link']}")
    print()  # Add a blank line between movies

Title: Inception
Reason: Directed by Christopher Nolan, like Interstellar, Inception offers a complex narrative blending action with deep psychological and philosophical themes. Its innovative storytelling and visual effects make it a compelling watch for fans of intellectually stimulating cinema.
Trailer Link: https://www.youtube.com/watch?v=YoHD9XEInc0

Title: Arrival
Reason: Arrival, a film directed by Denis Villeneuve, explores themes of communication and time, similar to Interstellar's exploration of space and time. It combines drama and sci-fi elements with a strong emotional core, making it a thought-provoking experience.
Trailer Link: https://www.youtube.com/watch?v=tFMo3UJ4B4g

Title: Gravity
Reason: Gravity is a visually stunning film that delves into the themes of survival and resilience in space, akin to Interstellar. Directed by Alfonso Cuarón, it offers breathtaking visuals and a gripping storyline, focusing on a more personal survival story.
Trailer Link: https://www.you

## Last example

In [24]:
genre = "horror"
favorite_movie_in_genre = "insidious"
age = "new"

# run chain
output = chain.invoke({'genre': genre, 'favorite_movie_in_genre': favorite_movie_in_genre, 'age': age})

output_content = output.content

In [25]:
parsed_movies = parse_movie_output(output_content)

for movie in parsed_movies:
    print(f"Title: {movie['title']}")
    print(f"Reason: {movie['reason']}")
    print(f"Trailer Link: {movie['trailer_link']}")
    print()  # Add a blank line between movies

Title: The Conjuring
Reason: Directed by James Wan, who also directed "Insidious," "The Conjuring" offers a similarly intense and atmospheric horror experience, focusing on paranormal investigators tackling a dark presence in a family home.
Trailer Link: https://www.youtube.com/watch?v=k10ETZ41q5o

Title: Sinister
Reason: "Sinister" features a storyline involving supernatural elements and a family in peril, much like "Insidious." Its use of suspense and the psychological depth of its horror are particularly gripping.
Trailer Link: https://www.youtube.com/watch?v=_kbQAJR9YWQ

Title: Hereditary
Reason: "Hereditary" offers a new twist on horror with its deeply unsettling atmosphere and focus on family tragedy and supernatural horror, pushing the boundaries of the genre much like "Insidious."
Trailer Link: https://www.youtube.com/watch?v=V6wWKNij_1M

